In [ ]:
import numpy as np
import pandas as pd

In [ ]:
raw_text = open('/content/pg5200.txt', encoding='utf-8-sig').read()

In [ ]:
raw_text[:100]

'The Project Gutenberg EBook of Metamorphosis, by Franz Kafka\nTranslated by David Wyllie.\n\nThis eBook'

In [ ]:
import re

In [ ]:
final = re.sub(r'\W+', ' ', raw_text).lower()

In [ ]:
final[:100]

'the project gutenberg ebook of metamorphosis by franz kafka translated by david wyllie this ebook is'

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
tokens = nltk.word_tokenize(final)

In [ ]:
tokens[:10]

['the',
 'project',
 'gutenberg',
 'ebook',
 'of',
 'metamorphosis',
 'by',
 'franz',
 'kafka',
 'translated']

In [ ]:
train_len = 3+1
text_sequences = []
for i in range(train_len,len(tokens)):
    seq = tokens[i-train_len:i]
    text_sequences.append(seq)

In [ ]:
text_sequences[:5]

[['the', 'project', 'gutenberg', 'ebook'],
 ['project', 'gutenberg', 'ebook', 'of'],
 ['gutenberg', 'ebook', 'of', 'metamorphosis'],
 ['ebook', 'of', 'metamorphosis', 'by'],
 ['of', 'metamorphosis', 'by', 'franz']]

In [ ]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)
sequences = tokenizer.texts_to_sequences(text_sequences) 

In [ ]:
sequences[:5]

[[1, 47, 43, 286],
 [47, 43, 286, 5],
 [43, 286, 5, 511],
 [286, 5, 511, 56],
 [5, 511, 56, 680]]

In [ ]:
sequences = np.array(sequences)

In [ ]:
train_inputs = sequences[:,:-1]
train_targets = sequences[:,-1]

In [ ]:
train_inputs[:5]

array([[  1,  47,  43],
       [ 47,  43, 286],
       [ 43, 286,   5],
       [286,   5, 511],
       [  5, 511,  56]])

In [ ]:
train_targets[:5]

array([286,   5, 511,  56, 680])

In [ ]:
len(tokenizer.word_index)

3015

In [ ]:
seq_len = train_inputs.shape[1]
vocabulary_size = len(tokenizer.word_index)+1

In [ ]:
from keras.utils import to_categorical
train_targets = to_categorical(train_targets, num_classes=vocabulary_size)

In [ ]:
np.argmax(train_targets[1])

5

In [ ]:
from keras.layers import Dense, LSTM, Embedding
from keras.layers import Dropout
from keras.models import Sequential

In [ ]:
model = Sequential()
model.add(Embedding(vocabulary_size, seq_len, input_length=seq_len))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(50,activation='relu'))
model.add(Dense(vocabulary_size, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 3)              9048      
                                                                 
 lstm (LSTM)                 (None, 3, 50)             10800     
                                                                 
 lstm_1 (LSTM)               (None, 50)                20200     
                                                                 
 dense (Dense)               (None, 50)                2550      
                                                                 
 dense_1 (Dense)             (None, 3016)              153816    
                                                                 
Total params: 196,414
Trainable params: 196,414
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(train_inputs,train_targets,epochs=100,verbose=1)

Epoch 1/100
802/802 [==============================] - 13s 7ms/step - loss: 6.4663 - accuracy: 0.0499
Epoch 2/100
802/802 [==============================] - 6s 7ms/step - loss: 6.1821 - accuracy: 0.0519
Epoch 3/100
802/802 [==============================] - 6s 7ms/step - loss: 6.1410 - accuracy: 0.0519
Epoch 4/100
802/802 [==============================] - 6s 7ms/step - loss: 6.0455 - accuracy: 0.0519
Epoch 5/100
802/802 [==============================] - 6s 7ms/step - loss: 5.9332 - accuracy: 0.0548
Epoch 6/100
802/802 [==============================] - 6s 7ms/step - loss: 5.7410 - accuracy: 0.0646
Epoch 7/100
802/802 [==============================] - 6s 7ms/step - loss: 5.5938 - accuracy: 0.0716
Epoch 8/100
802/802 [==============================] - 6s 7ms/step - loss: 5.4873 - accuracy: 0.0760
Epoch 9/100
802/802 [==============================] - 6s 7ms/step - loss: 5.3812 - accuracy: 0.0833
Epoch 10/100
802/802 [==============================] - 6s 7ms/step - loss: 5.2791 - accur

In [ ]:
from keras.utils import pad_sequences

In [ ]:
input_text = 'We have learned'
encoded_text = tokenizer.texts_to_sequences([input_text])[0]
pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
print(encoded_text, pad_encoded)

[85, 37, 613] [[ 85  37 613]]


In [ ]:
np.argmax(model.predict(pad_encoded))

1/1 [==============================] - 0s 15ms/step


38

In [ ]:
tokenizer.index_word[38]

'there'

In [ ]:
for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
  pred_word = tokenizer.index_word[i]
  print("Next word suggestion:",pred_word)

1/1 [==============================] - 0s 16ms/step
Next word suggestion: there
Next word suggestion: course
Next word suggestion: the
